In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import geopy
from geopy.geocoders import Nominatim

In [2]:
# df1 = pd.read_excel(r'..\data\raw\2024-09-01 Standard Coverage Model - JR.xlsx', sheet_name= 'Rep Addresses');
df1 = pd.read_excel('../data/raw/2024-09-01 Standard Coverage Model - JR.xlsx', sheet_name= 'Rep Addresses');

# df2 = pd.read_excel(r'C:\Users\filename.xlsx', sheet_name= 'Sheet2');

In [3]:
df1.head(5)

,Division,Region,Region\nName,Territory,Name,Last Name,* FAX # \nCELL #,Address_1,City,State,Zip,E-mail,Country,Comments,Regionals to Sort Separate,Notes
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN
3,A,H3,Allentown,R5,Brian,Ager,443-858-7970,1218 Fairfield Estates Lane,Crownsville,MD,21032,brian.ager@batesville.com,NaN,NaN,NaN,NaN
4,A,H4,Batesville,BB,Jason,Aldrin,* 773.631.9042\n773.505.0926,5225 N. New England Ave.,Chicago,IL,60656,jason.aldrin@batesville.com,NaN,NaN,NaN,NaN


In [4]:
df1.shape

(112, 16)

In [5]:
df1 = df1.rename(columns={'Division': 'division',
                          'Region': 'region_code',
                          'Territory' : 'territory',
                          'Name': 'name',
                          'Last Name' : 'last_name',
                          'Region\nName': 'region_name', 
                          ' * FAX # \nCELL #': 'fax/cell',
                         'Address_1' : 'street_address',
                          'City' : 'city',
                          'State' : 'state',
                          'Zip' : 'zip',
                          'E-mail' : 'email',
                          'Country' : 'country',
                          'Comments' : 'comments',
                          'Regionals to Sort Separate' : 'regionals_to_sort_separate',
                          'Notes' : 'notes'
                         })


In [6]:
df1.columns

Index(['division', 'region_code', 'region_name', 'territory', 'name',
       'last_name', 'fax/cell', 'street_address', 'city', 'state', 'zip',
       'email', 'country', 'comments', 'regionals_to_sort_separate', 'notes'],
      dtype='object')

In [7]:
df1

,division,region_code,region_name,territory,name,last_name,fax/cell,street_address,city,state,zip,email,country,comments,regionals_to_sort_separate,notes
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN
3,A,H3,Allentown,R5,Brian,Ager,443-858-7970,1218 Fairfield Estates Lane,Crownsville,MD,21032,brian.ager@batesville.com,NaN,NaN,NaN,NaN
4,A,H4,Batesville,BB,Jason,Aldrin,* 773.631.9042\n773.505.0926,5225 N. New England Ave.,Chicago,IL,60656,jason.aldrin@batesville.com,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df1 = df1.dropna(how='all')

In [9]:
df1['full_address'] = df1['street_address'].str.strip() + ', ' + df1['city'].str.strip() + ', ' + df1['state'].str.strip()

/var/folders/1p/m4db66qj4fgbpl1bn1vttcc80000gn/T/ipykernel_61901/974523268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['full_address'] = df1['street_address'].str.strip() + ', ' + df1['city'].str.strip() + ', ' + df1['state'].str.strip()


In [24]:
# df1.to_csv('../data/processed/rep-addresses.csv')

In [40]:
rep_addresses_df = pd.read_csv('../data/processed/rep-addresses.csv', index_col=0)

In [41]:
rep_addresses_df.columns

Index(['division', 'region_code', 'region_name', 'territory', 'name',
       'last_name', 'fax/cell', 'street_address', 'city', 'state', 'zip',
       'email', 'country', 'comments', 'regionals_to_sort_separate', 'notes',
       'full_address', 'lat', 'lon', 'formatted', 'district', 'name.1',
       'housenumber', 'street', 'suburb', 'postcode', 'city.1', 'county',
       'state.1', 'state_code', 'country.1', 'country_code', 'confidence',
       'confidence_city_level', 'confidence_street_level', 'attribution',
       'attribution_license', 'attribution_url'],
      dtype='object')

In [42]:
rep_addresses_df = rep_addresses_df.drop(['country.1', 'country_code','attribution', 'attribution_license', 'attribution_url'], axis=1)

In [43]:
rep_addresses_df = rep_addresses_df[rep_addresses_df.index < 103] 

In [45]:
rep_addresses_df.to_csv('../data/processed/rep-addresses-geocoded.csv')